<a href="https://colab.research.google.com/github/AsraniSanjana/PDL_winternship/blob/main/ColabFiles/gino_23st_dataset_EmptyModelErrorSolved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **install the libraries**

In [ ]:
!pip install plotly
!pip install lazypredict
!pip install m2cgen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 2.2 MB/s eta 0:00:00


# **upload the dataset files. the regex just concats all the files whose names end w a (2) and creates a new dataset.**

In [ ]:
import glob
import pandas as pd

csv_files = glob.glob("*(3).csv")
#csv_files = glob.glob("*w.csv")
print(csv_files)

dataset = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
dataset.to_csv("dataset_raw1.csv", index=False)

['10_wled_raw (3).csv', '50_wled_raw (3).csv', 'blank_wled_raw (3).csv', '500_wled_raw (3).csv', '100_wled_raw (3).csv', '200_wled_raw (3).csv', '20_wled_raw (3).csv']


In [ ]:
import pandas as pd
from lazypredict.Supervised import LazyClassifier
import plotly.express as px
from sklearn import model_selection, tree, metrics, ensemble
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('dataset_raw1.csv')
df = df.drop(['Clear1', 'Clear2'], axis=1)

# features= df.drop('class', axis=1)
# labels= df['class']

# classes = df["class"].unique().tolist()
# columns = features.columns.tolist()


# X_train, X_test, y_train, y_test = model_selection.train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
# df = df[['R1', 'G1', 'B1', 'Temperature1', 'Intensity1','R2', 'G2', 'B2', 'Temperature2', 'Intensity2']]

# Replace None values with NaN
import numpy as np

df.replace('None', np.nan, inplace=True)  # Assuming 'None' is represented as a string

# Convert columns to numeric (if needed)
df = df.apply(pd.to_numeric, errors='coerce')

# Replace NaN values with column means
df.fillna(df.mean(), inplace=True)

print(df)

       R1   G1   B1  Temperature1  Intensity1   R2   G2   B2  Temperature2  \
0     173  176  152       4738.51     1065.95  208  272  240       5809.08   
1     174  176  152       4719.28     1067.05  208  272  240       5809.08   
2     173  175  151       4716.49     1061.46  208  272  240       5810.60   
3     173  175  151       4716.49     1061.46  206  270  238       5815.21   
4     173  175  151       4716.49     1061.46  205  268  237       5818.32   
...   ...  ...  ...           ...         ...  ...  ...  ...           ...   
3611  106  112   79       4230.53      737.38   98  118  100       5321.95   
3612  102  107   76       4257.93      702.28  103  126  106       5315.87   
3613  103  109   77       4239.25      717.04  124  152  130       5393.10   
3614  103  109   77       4239.25      717.04  122  151  129       5393.91   
3615  109  117   82       4248.50      764.70   96  116  100       5365.76   

      Intensity2  class  
0        1522.05     10  
1        15

In [ ]:
features= df.drop('class', axis=1)
labels= df['class']

classes = df["class"].unique().tolist()
columns = features.columns.tolist()


X_train, X_test, y_train, y_test = model_selection.train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
X_train


,R1,G1,B1,Temperature1,Intensity1,R2,G2,B2,Temperature2,Intensity2
2592,193,151,104,3463.80,1054.55,204,281,262,6311.84,1508.67
3216,110,120,81,4196.54,799.19,130,163,125,5048.60,982.63
1228,13,18,19,6959.46,91.48,14,19,21,7334.60,85.10
1234,13,18,19,6959.46,91.48,14,19,21,7334.60,85.10
794,129,227,237,8390.77,1132.76,220,326,307,6213.03,898.31
...,...,...,...,...,...,...,...,...,...,...
1130,13,18,19,6959.46,91.48,14,19,21,7334.60,85.10
1294,13,18,19,6959.46,91.48,14,19,21,7334.60,85.10
860,95,152,148,7326.58,800.90,130,179,161,6142.12,988.35
3507,108,115,78,4142.67,767.38,114,142,114,5201.00,840.55


In [ ]:
X_test.describe

<bound method NDFrame.describe of        R1   G1   B1  Temperature1  Intensity1   R2   G2   B2  Temperature2  \
2917  136  108   79       3604.16      738.03   98  112   98       5201.00   
1255   13   18   19       6959.46       91.48   14   19   21       7334.60   
2800  210  163  114       3459.29     1137.83  198  253  228       5797.87   
2144  118  147  184       7332.02      655.65  150  208  199       6515.72   
1936   81   93   82       5248.04      545.79  102  135  123       6009.18   
...   ...  ...  ...           ...         ...  ...  ...  ...           ...   
176   167  165  142       4630.64     1006.40  178  230  205       5800.82   
1953   96  108   98       5280.38      635.18  117  161  146       6223.60   
422    85   80   66       4349.35      502.59  103  115   95       4897.71   
680    97  180  191       8816.61      876.11  175  260  243       6775.19   
2588  142  112   81       3564.31      777.81  134  188  176       6446.29   

      Intensity2  
2917      

In [ ]:
df["class"].unique()

array([ 10,  50,   0, 500, 100, 200,  20])

In [ ]:
df['class'].value_counts()

200    526
500    525
10     516
100    515
20     512
50     511
0      511
Name: class, dtype: int64

In [ ]:
df['class'].replace('blank', 0, inplace=True)
classes = df["class"].unique().tolist()
# classes.sort()
print(classes)   # Create classes list in pico program

[0, 10, 20, 50, 100, 200, 500]


In [ ]:
# classes = df['class'].unique().tolist()
# classes.sort()

# for cls in classes:
#     print((cls, 200))


In [ ]:
print(classes)
print(columns)
print(labels.value_counts())


[0, 10, 20, 50, 100, 200, 500]
['R1', 'G1', 'B1', 'Temperature1', 'Intensity1', 'R2', 'G2', 'B2', 'Temperature2', 'Intensity2']
200    526
500    525
10     516
100    515
20     512
50     511
0      511
Name: class, dtype: int64


# **visualize it**

In [ ]:
# Convert class column from numerical to strings or categorical
df['class'] = df['class'].astype(str)

fig = px.scatter_3d(df, x='R1', y='G1', z='B1', color='class')

fig.update_layout(scene=dict(
                    xaxis_title='R1',
                    yaxis_title='G1',
                    zaxis_title='B1'),
                    margin=dict(l=0, r=0, b=0, t=0))
fig.show()


In [ ]:
clf1 = LazyClassifier()

predictions, scores = clf1.fit(X_train, X_test, y_train, y_test)
predictions

 90%|████████▉ | 26/29 [00:04<00:00,  6.98it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000516 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2171
[LightGBM] [Info] Number of data points in the train set: 2892, number of used features: 10
[LightGBM] [Info] Start training from score -1.955988
[LightGBM] [Info] Start training from score -2.006124
[LightGBM] [Info] Start training from score -1.941425
[LightGBM] [Info] Start training from score -1.924698
[LightGBM] [Info] Start training from score -1.939018
[LightGBM] [Info] Start training from score -1.929449
[LightGBM] [Info] Start training from score -1.927071
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

100%|██████████| 29/29 [00:06<00:00,  4.77it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LabelPropagation,1.00,1.00,None,1.00,0.40
LabelSpreading,1.00,1.00,None,1.00,0.73
RandomForestClassifier,1.00,1.00,None,1.00,0.53
ExtraTreesClassifier,1.00,1.00,None,1.00,0.27
LGBMClassifier,1.00,1.00,None,1.00,1.15
LogisticRegression,0.99,0.99,None,0.99,0.11
SVC,0.99,0.99,None,0.99,0.10
DecisionTreeClassifier,0.99,0.99,None,0.99,0.03
KNeighborsClassifier,0.99,0.99,None,0.99,0.06


In [ ]:
predictions

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LabelPropagation,1.00,1.00,None,1.00,0.40
LabelSpreading,1.00,1.00,None,1.00,0.73
RandomForestClassifier,1.00,1.00,None,1.00,0.53
ExtraTreesClassifier,1.00,1.00,None,1.00,0.27
LGBMClassifier,1.00,1.00,None,1.00,1.15
LogisticRegression,0.99,0.99,None,0.99,0.11
SVC,0.99,0.99,None,0.99,0.10
DecisionTreeClassifier,0.99,0.99,None,0.99,0.03
KNeighborsClassifier,0.99,0.99,None,0.99,0.06


#**training**

In [ ]:
import sklearn
import sys
sys.setrecursionlimit(999999999)
import m2cgen as m2c

In [ ]:
DTC= sklearn.tree.DecisionTreeClassifier()
DTC.fit(X_train, y_train)

input = np.array([161	, 126,	89,	3497.15,	874.943,	182,	253,	235, 6358.19,	1355.25]).reshape(1,-1)
pred = DTC.predict(input)
print("prediction result: {}".format(pred))

code_py = m2c.export_to_python(DTC, function_name="classify")
with open("DTC.py", "w") as fp:
  fp.write(code_py)

!python-minimizer DTC.py -o DTC.py


prediction result: [200]
/bin/bash: line 1: python-minimizer: command not found


In [ ]:
ETsC= sklearn.ensemble.ExtraTreesClassifier()
ETsC.fit(X_train, y_train)

input = np.array([161	, 126,	89,	3497.15,	874.943,	182,	253,	235, 6358.19,	1355.25]).reshape(1,-1)
pred = ETsC.predict(input)
print("prediction result: {}".format(pred))


code_py = m2c.export_to_python(ETsC, function_name="classify")
with open("ETsC.py", "w") as fp:
  fp.write(code_py)

!python-minimizer ETsC.py -o ETsC.py


prediction result: [200]
/bin/bash: line 1: python-minimizer: command not found


In [ ]:
LinearSVC= sklearn.svm.LinearSVC()
LinearSVC.fit(X_train, y_train)

input = np.array([161	, 126,	89,	3497.15,	874.943,	182,	253,	235, 6358.19,	1355.25]).reshape(1,-1)
pred = LinearSVC.predict(input)
print("prediction result: {}".format(pred))


code_py = m2c.export_to_python(LinearSVC, function_name="classify")
with open("LinearSVC.py", "w") as fp:
  fp.write(code_py)

!python-minimizer LinearSVC.py -o LinearSVC.py



prediction result: [200]
/bin/bash: line 1: python-minimizer: command not found


In [ ]:
LogReg_clf = sklearn.linear_model.LogisticRegression()
LogReg_clf.fit(X_train, y_train)

input = np.array([161	, 126,	89,	3497.15,	874.943,	182,	253,	235, 6358.19,	1355.25]).reshape(1,-1)
pred = LogReg_clf.predict(input)
print("prediction result: {}".format(pred))


code_py = m2c.export_to_python(LogReg_clf, function_name="classify")
with open("LogReg_clf.py", "w") as fp:
  fp.write(code_py)

!python-minimizer LogReg_clf.py -o LogReg_clf.py

prediction result: [200]
/bin/bash: line 1: python-minimizer: command not found


In [ ]:
RFC= sklearn.ensemble.RandomForestClassifier(n_estimators=100)

RFC.fit(X_train, y_train)

input = np.array([161	, 126,	89,	3497.15,	874.943,	182,	253,	235, 6358.19,	1355.25]).reshape(1,-1)
pred = RFC.predict(input)
print("prediction result: {}".format(pred))


code_py = m2c.export_to_python(RFC, function_name="classify")
with open("RFC.py", "w") as fp:
  fp.write(code_py)

!python-minimizer RFC.py -o RFC.py

prediction result: [200]
/bin/bash: line 1: python-minimizer: command not found


the below code functionality is to be replicated in ckt python. after printing predictions of all classes, predn for which max models resulted in has to be printed.

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

# Initialize classifiers
classifiers = [
    DecisionTreeClassifier(),
    ExtraTreesClassifier(),
    LinearSVC(),
    LogisticRegression(),
    RandomForestClassifier(n_estimators=100)
]

# Train each classifier and make predictions
all_predictions = []
for clf in classifiers:
    clf.fit(X_train, y_train)
    input_query = np.array([173	,176,	151	,	4716.49,	1061.46,	207,	270,	238,	5793.03,	1511.98]).reshape(1, -1)
    prediction = clf.predict(input_query)
    all_predictions.append(prediction)

    # Print individual model prediction for the input query
    print(f"{clf.__class__.__name__} prediction: {prediction}")

# Stack predictions vertically
stacked_predictions = np.vstack(all_predictions)

# Perform majority voting for the input query
final_prediction = np.argmax(np.bincount(stacked_predictions.flatten()))

# Print final prediction for the input query
print(f"Final prediction for the input query: {final_prediction}")


DecisionTreeClassifier prediction: [10]
ExtraTreesClassifier prediction: [10]
LinearSVC prediction: [10]
LogisticRegression prediction: [10]
RandomForestClassifier prediction: [10]
Final prediction for the input query: 10


In [ ]:
import DTC, ETsC, LinearSVC, LogReg_clf,RFC

pred =DTC.classify([173	,176,	151	,	4716.49,	1061.46,	207,	270,	238,	5793.03,	1511.98])

if pred is not None:
  pred_idx = pred.index(max(pred))
  pred_d = classes[pred_idx]
  print("class: ",pred_d)


class:  10


In [ ]:
import DTC
import ETsC
import LinearSVC
import LogReg_clf
import RFC
import numpy as np

# Example input query
# input_query = np.array([126,	229,	230,	8376,	1162.63,	222,	313,	279,	'None',	'None'])
input_query = np.array([173, 176, 151, 4716.49, 1061.46, 207, 270, 238, 5793.03, 1511.98])

# Define the list of models and their names
models = [
    ("DecisionTreeClassifier", DTC),
    ("ExtraTreesClassifier", ETsC),
    ("LinearSVC", LinearSVC),
    ("LogisticRegression", LogReg_clf),
    ("RandomForestClassifier", RFC)
]

# Call the classify method from each model
all_predictions = []
for model_name, model in models:
    pred = model.classify(input_query)
    all_predictions.append(pred)

    # Print the predicted class by each model
    if pred is not None:
        pred_idx = np.argmax(pred)
        pred_d = classes[pred_idx] if 'classes' in globals() else f"Class {pred_idx}"
        print(f"Predicted class by {model_name}: {pred_d}")

# Count occurrences of each class across all models
class_counts = np.sum(all_predictions, axis=0)

# Get the class with the maximum count
max_class_index = np.argmax(class_counts)

# Print the final predicted class
if 'classes' in globals():
    print(f"Final predicted class across all models: {classes[max_class_index]}")
else:
    print("Global variable 'classes' not defined.")


Predicted class by DecisionTreeClassifier: 10
Predicted class by ExtraTreesClassifier: 10
Predicted class by LinearSVC: 10
Predicted class by LogisticRegression: 10
Predicted class by RandomForestClassifier: 10
Final predicted class across all models: 10


# **how to handle if t and i are None** dataset to which these tuples belong to: 50 and 20: creating a new dt model, with only 2 rgb values as features.

157,	283,	296, None,	None,	218,	334,	318,	None,	None  ..........t1i1 & t2i2 both none

input: 157,	283,	296,	218,	334,	318


---

if any one of the 2 pairs is none:

163,	296,	309,	None,	None,	187,	279,	266, 6854.79,	1502.13.....t1i1 none

input: 163,	296,	309,	187,	279,	266


---


126,	229,	230, 8376,	1162.63,	222,	313,	279, None, None ..........t2i2 none

input: 126,	229,	230,	222,	313,	279

---




In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

# Assuming 'df' is your dataset DataFrame with columns R1, G1, B1, R2, G2, B2, class

# Extract features and target variable
features = df[['R1', 'G1', 'B1', 'R2', 'G2', 'B2']]
target = df['class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Train a model using only R1, G1, B1, R2, G2, B2 values
rgb_dt= DecisionTreeClassifier()
rgb_dt.fit(X_train, y_train)

# Make predictions on the test set
rgb_predictions = rgb_dt.predict(X_test)

# Evaluate the model
accuracy_rgb = accuracy_score(y_test, rgb_predictions)
print(f"Accuracy of RGB-only model: {accuracy_rgb:.2f}")

# Example: Test the model with a new query
new_query = pd.DataFrame([[126,	229,	230,	222,	313,	279]], columns=['R1', 'G1', 'B1', 'R2', 'G2', 'B2'])
predicted_class = rgb_dt.predict(new_query)

print(f"Predicted class for the new query: {predicted_class[0]}")


Accuracy of RGB-only model: 0.98
Predicted class for the new query: 50


In [ ]:
code_py = m2c.export_to_python(rgb_dt, function_name="classify")
with open("rgb_dt.py", "w") as fp:
  fp.write(code_py)

code_py
!python-minimizer rgb_dt.py -o rgb_dt.py


/bin/bash: line 1: python-minimizer: command not found


In [ ]:
import importlib
import rgb_dt
rgb_dt = importlib.reload(rgb_dt)
pred = rgb_dt.classify([126,	229,	230,	222,	313,	279])
# pred = rgb_dt.classify([157,	283,	296,	218,	334,	318])
#pred = rgb_dt.classify([175,	204,	137,	204,	285,254])  #20
print(pred)
print(classes)
if pred is not None:
  pred_idx = pred.index(max(pred))
  pred_d = classes[pred_idx]
  print("class: ",pred_d)


[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
[0, 10, 20, 50, 100, 200, 500]
class:  200


In [ ]:
!python-minimizer DTC.py -o compressed_DTC.py

/bin/bash: line 1: python-minimizer: command not found
